In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from src.data_loading import DlDataset
from src.convnet import train_net, NumNet

import matplotlib.pyplot as plt

In [2]:
# Not very elegant but removes all the warnings about deprecated upsample funtion
import warnings
warnings.filterwarnings("ignore")

In [3]:
#import data for a 1000 pairs
N = 1000
dataset_train = DlDataset(N, normalize=True, upsample=(28,28))
dataset_test = DlDataset(N, normalize=True, upsample=(28,28))
dataset_train.train()
dataset_test.test()

In [ ]:
#model, simple cnn with 2 conv layers and one hidden and output layer
#Number of parameters 224290
class Net(nn.Module):
    def __init__(self,hiddenUnits):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64*5*5, hiddenUnits)
        self.fc2 = nn.Linear(hiddenUnits, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.drop1 = nn.Dropout2d(p=0.25)
        self.drop2 = nn.Dropout(p=0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2)
        x = self.bn1(x)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2)
        #x = self.drop1(x)
        x = self.bn2(x)        
        x = F.relu(self.fc1(x.view(-1, 64*5*5)))
        #x = self.drop2(x)
        #x = self.fc2(x)
        x = F.log_softmax(self.fc2(x),dim=1)
        return x

In [4]:
# The same net using NumNet class
# Dropout layer
# {'Type':'DropoutLayer', 'p':0.2}
config = [{'Type': 'ConvLayer', 'out_channels':32, 'activation':'ReLU()', 'kernel_size':5},
          {'Type': 'MaxPoolLayer', 'pooling':2, 'stride':2},
          {'Type': 'BatchNormLayer'},
          {'Type': 'ConvLayer', 'out_channels':64, 'activation':'ReLU()', 'kernel_size':3},
          {'Type': 'MaxPoolLayer', 'pooling':2, 'stride':2},
          {'Type': 'BatchNormLayer'},
          {'Type': 'LinearLayer', 'out_features':128, 'activation':'ReLU()'},
          {'Type': 'LinearLayer', 'out_features':2, 'activation':'LogSoftmax(dim=1)'}]

model = NumNet([2,28,28], config)
model.summary()

Number              Layer               Output shape        Input shape         Trainable params    
0                   Conv2d              [2, 28, 28]         [32, 24, 24]        1632
1                   MaxPool2d           [32, 24, 24]        [32, 12, 12]        0
2                   BatchNorm2d         [32, 12, 12]        [32, 12, 12]        64
3                   Conv2d              [32, 12, 12]        [64, 10, 10]        18496
4                   MaxPool2d           [64, 10, 10]        [64, 5, 5]          0
5                   BatchNorm2d         [64, 5, 5]          [64, 5, 5]          128
6                   Linear              1600                128                 204928
7                   Linear              128                 2                   258
----------------------------------------------------------------------------------------------------
Total number of trainable parameters: 225506


In [ ]:
# Train parameter
epochs = 15
eta = 5e-3
lambda_l2 = 1e-3
gamma = 0.2
depth = 4
n_filters = 3

# U-Net definition
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu') #Hannes' gpu is not supported but has cuda cores... 
#model = Net(128).to(device)
optim = torch.optim.SGD(model.parameters(), lr=eta, momentum=gamma)
criterion = torch.nn.CrossEntropyLoss()
dataloader = []
dataloader.append(DataLoader(dataset_train, batch_size=10,shuffle=True, num_workers=4))
dataloader.append(DataLoader(dataset_test, batch_size=10,shuffle=True, num_workers=4))

avg_loss_train,avg_acc_train,avg_loss_test,avg_acc_test, model = train_net(model, device, optim, criterion, dataloader,
                                                     epochs=epochs, lambda_=1e-3, reg_type=None, 
                                                     save=False)

In [ ]:
plt.figure('History Plot', figsize=(12,8))
plt.title('Error and Accuracy of Network')
plt.subplot(1,2,1)
plt.plot(range(0,epochs), avg_loss_train, label = 'Train error')
plt.plot(range(0,epochs), avg_loss_test, label = 'Validation error')
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(range(0,epochs), avg_acc_train, label = 'Train accuracy')
plt.plot(range(0,epochs), avg_acc_test, label = 'Validation accuracy')
plt.grid()
plt.legend();

In [ ]:
dataset_test.infere(model,5);
dataset_test.infere(model,6);